In [1]:
import tensorflow as tf
import numpy as np
import os

/Users/ruizheng/anaconda3/envs/carnd-term1/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)
/Users/ruizheng/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## use .np file

In [12]:
sess = tf.Session()
model_dir = '../vgg_model'
vgg_path = os.path.join(model_dir, 'vgg')
tf.saved_model.loader.load(sess, ["vgg16"], vgg_path)
graph = tf.get_default_graph()
image_input = graph.get_tensor_by_name('image_input:0')
keep_prob = graph.get_tensor_by_name('keep_prob:0')
layer3 = graph.get_tensor_by_name('layer3_out:0')
layer4 = graph.get_tensor_by_name('layer4_out:0')
layer7 = graph.get_tensor_by_name('layer7_out:0')

INFO:tensorflow:Restoring parameters from ../vgg_model/vgg/variables/variables


In [5]:
[n.name for n in tf.get_default_graph().as_graph_def().node]

['image_input',
 'keep_prob',
 'Processing/split/split_dim',
 'Processing/split',
 'Processing/sub/y',
 'Processing/sub',
 'Processing/sub_1/y',
 'Processing/sub_1',
 'Processing/sub_2/y',
 'Processing/sub_2',
 'Processing/concat/axis',
 'Processing/concat',
 'conv1_1/filter/Initializer/Const',
 'conv1_1/filter',
 'conv1_1/filter/Assign',
 'conv1_1/filter/read',
 'conv1_1/L2Loss',
 'conv1_1/weight_loss/y',
 'conv1_1/weight_loss',
 'conv1_1/Conv2D',
 'conv1_1/biases/Initializer/Const',
 'conv1_1/biases',
 'conv1_1/biases/Assign',
 'conv1_1/biases/read',
 'conv1_1/BiasAdd',
 'conv1_1/Relu',
 'conv1_2/filter/Initializer/Const',
 'conv1_2/filter',
 'conv1_2/filter/Assign',
 'conv1_2/filter/read',
 'conv1_2/L2Loss',
 'conv1_2/weight_loss/y',
 'conv1_2/weight_loss',
 'conv1_2/Conv2D',
 'conv1_2/biases/Initializer/Const',
 'conv1_2/biases',
 'conv1_2/biases/Assign',
 'conv1_2/biases/read',
 'conv1_2/BiasAdd',
 'conv1_2/Relu',
 'pool1',
 'conv2_1/filter/Initializer/Const',
 'conv2_1/filter',
 

In [13]:
reuse_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
reuse_vars

[<tf.Variable 'conv1_1/kernel:0' shape=(3, 3, 3, 64) dtype=float32_ref>,
 <tf.Variable 'conv1_1/bias:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'conv1_2/kernel:0' shape=(3, 3, 64, 64) dtype=float32_ref>,
 <tf.Variable 'conv1_2/bias:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'conv1_1/filter:0' shape=(3, 3, 3, 64) dtype=float32_ref>,
 <tf.Variable 'conv1_1/biases:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'conv1_2/filter:0' shape=(3, 3, 64, 64) dtype=float32_ref>,
 <tf.Variable 'conv1_2/biases:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'conv2_1/filter:0' shape=(3, 3, 64, 128) dtype=float32_ref>,
 <tf.Variable 'conv2_1/biases:0' shape=(128,) dtype=float32_ref>,
 <tf.Variable 'conv2_2/filter:0' shape=(3, 3, 128, 128) dtype=float32_ref>,
 <tf.Variable 'conv2_2/biases:0' shape=(128,) dtype=float32_ref>,
 <tf.Variable 'conv3_1/filter:0' shape=(3, 3, 128, 256) dtype=float32_ref>,
 <tf.Variable 'conv3_1/biases:0' shape=(256,) dtype=float32_ref>,
 <tf.Variable 'conv3_2/fi

## use .npz file

In [24]:
import numpy as np
weights = np.load("../vgg/vgg16_weights.npz")

In [34]:
class airbus_model():
    def __init__(self, image_shape=(768,768), model_dir=None, learning_rate = 0.0001, continue_training=False, vgg_path="../vgg_model/vgg"):
        self.image_shape = image_shape
        self.model_dir = model_dir
        self.lr = learning_rate
        self.continue_training = continue_training
        self.sess = tf.Session()
        
        self.x_holder = tf.placeholder(tf.float32, [None, self.image_shape[0], self.image_shape[1], 3], name="x_holder")
        self.y_holder = tf.placeholder(tf.float32, [None, self.image_shape[0], self.image_shape[1]], name="y_holder")
    
        self.conv_layers()
        
        
    def conv_layers(self):
        ############   down ##############
        # build vgg
        conv1_1 = tf.layers.conv2d(self.x_holder, filters=64, kernel_size=(3,3), padding='same', activation=tf.nn.relu, trainable=False, name='conv1_1')
        conv1_2 = tf.layers.conv2d(conv1_1, filters=64, kernel_size=(3,3), padding='same', activation=tf.nn.relu, trainable=False, name='conv1_2')
        pool1 = tf.layers.max_pooling2d(conv1_2, pool_size=(2,2), strides=2, padding='same', name='pool1')

        conv2_1 = tf.layers.conv2d(pool1, filters=128, kernel_size=(3,3), padding='same', activation=tf.nn.relu, trainable=False, name='conv2_1')
        conv2_2 = tf.layers.conv2d(conv2_1, filters=128, kernel_size=(3,3), padding='same', activation=tf.nn.relu, trainable=False, name='conv2_2')
        pool2 = tf.layers.max_pooling2d(conv2_2, pool_size=(2,2), strides=2, padding='same', name='pool2')


        conv3_1 = tf.layers.conv2d(pool2, filters=256, kernel_size=(3,3), padding='same', activation=tf.nn.relu, trainable=False, name='conv3_1')
        conv3_2 = tf.layers.conv2d(conv3_1, filters=256, kernel_size=(3,3), padding='same', activation=tf.nn.relu, trainable=False, name='conv3_2')
        conv3_3 = tf.layers.conv2d(conv3_2, filters=256, kernel_size=(3,3), padding='same', activation=tf.nn.relu, trainable=False, name='conv3_3')
        pool3 = tf.layers.max_pooling2d(conv3_3, pool_size=(2,2), strides=2, padding='same', name='pool3')


        conv4_1 = tf.layers.conv2d(pool3, filters=512, kernel_size=(3,3), padding='same', activation=tf.nn.relu, trainable=False, name='conv4_1')
        conv4_2 = tf.layers.conv2d(conv4_1, filters=512, kernel_size=(3,3), padding='same', activation=tf.nn.relu, trainable=False, name='conv4_2')
        conv4_3 = tf.layers.conv2d(conv4_2, filters=512, kernel_size=(3,3), padding='same', activation=tf.nn.relu, trainable=False, name='conv4_3')
        pool4 = tf.layers.max_pooling2d(conv4_3, pool_size=(2,2), strides=2, padding='same', name='pool4')


        conv5_1 = tf.layers.conv2d(pool4, filters=512, kernel_size=(3,3), padding='same', activation=tf.nn.relu, trainable=False, name='conv5_1')
        conv5_2 = tf.layers.conv2d(conv5_1, filters=512, kernel_size=(3,3), padding='same', activation=tf.nn.relu, trainable=False, name='conv5_2')
        conv5_3 = tf.layers.conv2d(conv5_2, filters=512, kernel_size=(3,3), padding='same', activation=tf.nn.relu, trainable=False, name='conv5_3')
        pool5 = tf.layers.max_pooling2d(conv5_3, pool_size=(2,2), strides=2, padding='same', name='pool5')
        
        #print(self.sess.run('conv1_1:0'))
        
        self.sess.run(tf.global_variables_initializer())
        self.load_weights()
        print(self.sess.run("conv1_1:0"))
              
    def load_weights(self, weight_file="../vgg/vgg16_weights.npz"):
        weights = np.load(weight_file)
        d = {var.name:var for var in tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)}
        #print(d.keys())
        for new_name in d.keys():
            #print(new_name)
            # observe d.keys() and weights.keys() to see the difference in naming
            if new_name.endswith('/kernel:0'):
                old_name = new_name[:7]+'_W'
            elif new_name.endswith('/bias:0'):
                old_name = new_name[:7]+'_b'
            d[new_name].assign_add(weights[old_name])
            #print(old_name)
            
model = airbus_model()

                    


AttributeError: 'airbus_model' object has no attribute 'graph'

In [21]:
print(model.sess.run("conv1_1:0"))

ValueError: Fetch argument 'conv1_1:0' cannot be interpreted as a Tensor. ("The name 'conv1_1:0' refers to a Tensor which does not exist. The operation, 'conv1_1', does not exist in the graph.")

In [58]:
for new_name in new_names:
weights = np.load("../vgg/vgg16_weights.npz")
    #print(new_name)
    if new_name.endswith('/kernel:0'):
        old_name = new_name[:7]+'_W'
    elif new_name.endswith('/bias:0'):
        old_name = new_name[:7]+'_b'
    d[new_name].assign_add(weights[old_name])
    #print(old_name)

## use .ckpt file

In [31]:
sess = tf.Session()
saver = tf.train.Saver()
saver.restore(sess, "../vgg/vgg_16.ckpt")

INFO:tensorflow:Restoring parameters from ../vgg/vgg_16.ckpt


NotFoundError: Tensor name "conv1_1/bias" not found in checkpoint files ../vgg/vgg_16.ckpt
	 [[Node: save_2/RestoreV2 = RestoreV2[dtypes=[DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, ..., DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT], _device="/job:localhost/replica:0/task:0/device:CPU:0"](_arg_save_2/Const_0_0, save_2/RestoreV2/tensor_names, save_2/RestoreV2/shape_and_slices)]]

Caused by op 'save_2/RestoreV2', defined at:
  File "/Users/ruizheng/anaconda3/envs/carnd-term1/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/ruizheng/anaconda3/envs/carnd-term1/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/ruizheng/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/ruizheng/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    ap

`it doesn't work, perhaps we also need ckpt.meta file`